In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

sns.set(rc={'figure.figsize':(15,8)})

path_to_catalog=r'C:\Users\kgrzebien\Desktop\python_tasks\reklamacje\raport_felery\\'


In [18]:
df_slownik=pd.read_csv('../słowniki/slownik.csv',
                       encoding='Cp1250',
                      sep="|")
df_slownik=df_slownik.loc[df_slownik['IS_DELETED']=='N'][['KOD_TEMA', 'NAZWA','RODZAJ','ZNACZNIK', 'TRZY', 'NAZWA_IMP', 'CENA_WAL', 'SYMBOL_IMP']]
df_slownik_znacznik=pd.read_csv('../słowniki/znacznik.csv',
                       encoding='Cp1250',
                      sep="|")
df_slownik_znacznik=df_slownik_znacznik.loc[df_slownik_znacznik['IS_DELETED']=='N'][['NAZWA','ZNACZNIK']]
df_slownik_znacznik=df_slownik_znacznik.rename(columns={'NAZWA':'NZNACZ'})

In [56]:
def func_read_data(path):
    df=pd.read_csv(path+'.csv',
              encoding='Cp1250',
              sep="|")
    return df
    
def func_str_to_float(s):
    s=s.astype(str)
    s=s.apply(lambda x: x.replace(',','.'))
    s=s.astype(float)
    return s

def func_add_time(df):
    
    df['Z_DNIA']=pd.to_datetime(df['Z_DNIA'], format='%y/%m/%d')
    
    df['year']=df['Z_DNIA'].apply(lambda x: x.year)
    df['month']=df['Z_DNIA'].apply(lambda x: x.month)
    
    #df=df.loc[df['Z_DNIA']<=np.datetime64('2021-05-13')]
    
    # df['day_of_year']=df['Z_DNIA'].apply(lambda x: x.timetuple().tm_yday)
    # df['week']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[1])
    # df['week_day']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[2])
    # df['year_month']=df['year'].astype(str)+'-'+df['month'].astype(str)+'-'+'01'
    # df['year_month']=df['year_month'].apply(lambda x: x[:5]+'0'+x[5:] if len(x)==9 else x)
    # df['year_month']=pd.to_datetime(df['year_month'], format='%Y-%m-%d')
    # df['year_week']=df['year'].astype(str)+'--'+df['week'].astype(str)
    # df['year_week']=df['year_week'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)==7 else x)
    # df['year_day']=df['year'].astype(str)+'--'+df['day_of_year'].astype(str)
    # df['year_day']=df['year_day'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)<9 else x)
    return df

def func_correct_name(df, name_col):
    list_of_name=['ZESTAW UPOMIN.']
    list_of_correct_name=['ZESTAWUPOMIN.']
    for zla_nazwa, dobra_nazwa in zip(list_of_name, list_of_correct_name):
        df[name_col]=df[name_col].replace(zla_nazwa, dobra_nazwa, regex=False)
    return df

In [34]:

    
def func_filtr_feler2(df, name_col):
    df['czy_z_f']=df[name_col].apply(lambda x: 'tak' if x[:2]=='F/' else 'nie')
    df['name_tmp']=df[name_col].apply(lambda x: x[2:] if x[:2]=='F/' else x)
    df_tmp_f=df.loc[df['czy_z_f']=='tak']
    df=df.loc[df['czy_z_f']=='nie']
    display(len(df_tmp_f))
    # display(df_tmp_f.loc[df_tmp_f['name_tmp']=='ZESTAWUPOMIN.'])
    # display(df.loc[df['name_tmp']=='ZESTAWUPOMIN.'])
    df['czy_jest_taki_felere']='nie'
    df_tmp_f['czy_dodany']='nie'
    df_tmp_f['NOWA_NAZWA']=''
    df_tmp_f['NOWA_RODZAJ']=''
    df_tmp_f['NOWA_KOD_TEMA']=''
    for name, rodzaj, ind in tqdm(zip(df_tmp_f['name_tmp'], df_tmp_f['RODZAJ1'], df_tmp_f.index)):
        if df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].empty:
            continue
        else:
            df_tmp_f.loc[ind, 'NOWA_NAZWA']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index][name_col].values[0]
            df_tmp_f.loc[ind, 'NOWA_RODZAJ']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index]['RODZAJ1'].values[0]
            df_tmp_f.loc[ind, 'NOWA_KOD_TEMA']=df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index]['KOD_TEMA'].values[0]
            # df.loc[df.loc[df[name_col].str.contains(name)].loc[df['RODZAJ1']==rodzaj].index, 'czy_jest_taki_felere']='tak'
            # df_tmp_f.loc[ind, 'czy_dodany']='tak'
     
    # df=df.loc[df['czy_jest_taki_felere']=='tak']
    # display(len(df))
    # df=df.append(df_tmp_f.loc[df_tmp_f['czy_dodany']=='nie'], ignore_index=True)
    return df_tmp_f.reset_index().drop('index', axis=1)

In [51]:
def func_raport(df, df_slownik, df_slownik_znacznik):
    df=df.merge(df_slownik_znacznik, how='left', on='ZNACZNIK')
    df=df.rename(columns={'NOWA_NAZWA':'NAZWA',
                          'NOWA_RODZAJ':'RODZAJ',
                          'ILOSC2':'ILOSC'})
    for i in tqdm(df.index):
        if df.loc[i,'NAZWA']=='':
            df.loc[i,'NAZWA']=df.loc[i,'NAZWA1']
            df.loc[i,'RODZAJ']=df.loc[i,'RODZAJ1']
    return df[['ZNACZNIK','NZNACZ','Z_DNIA','NAZWA','RODZAJ','SYMBOL_IMP','NAZWA_IMP', 'TRZY', 'ILOSC', 'CENA_WAL']]

In [4]:
### to jest zle bo kolumny z nazwami maja tylko 16 znakow

# def func_filtr_feler(df, name_col):
#     df['name_tmp']=df[name_col].apply(lambda x: x[2:] if x[:2]=='F/' else x)
#     df['czy_z_f']=df[name_col].apply(lambda x: 'tak' if x[:2]=='F/' else 'nie')
    
#     df_tmp=df.groupby(['FAKTURA','name_tmp']).agg({name_col:'nunique'}).reset_index()
#     df_tmp=df_tmp.loc[df_tmp[name_col]==2].drop(name_col, axis=1)
#     df=df.merge(df_tmp, how='inner', on=['FAKTURA','name_tmp'])
#     df=df.loc[df['czy_z_f']=='tak'].reset_index().drop('index', axis=1)
#     return df

In [29]:
df_test=pd.read_excel('raport_felery/FELERY2020_05_2021.xlsx')
#df_test=df_test.loc[df_test['ZNACZNIK']==19]
df_test['name_with_f']=df_test['NAZWA'].apply(lambda x: x if x[:2]=='F/' else 'F/'+x)
df_test['name_without_f']=df_test['NAZWA'].apply(lambda x: x[2:] if x[:2]=='F/' else x)


In [77]:
df=func_read_data('raport_felery/f56szac_21_06_22_06')
df['CENA_Z1']=func_str_to_float(df['CENA_Z1'])
df['CENA_Z2']=func_str_to_float(df['CENA_Z2'])
df['ILOSC1']=func_str_to_float(df['ILOSC1'])
df['ILOSC2']=func_str_to_float(df['ILOSC2'])
df['CENA_WAL']=func_str_to_float(df['CENA_WAL'])
df['CENA']=func_str_to_float(df['CENA'])
df=func_add_time(df)
df=func_correct_name(df, 'NAZWA1')

df_belki=func_read_data('raport_felery/f60szac_21_06_22_06')
df_belki['CENA_Z1']=func_str_to_float(df_belki['CENA_Z1'])
df_belki['CENA_Z2']=func_str_to_float(df_belki['CENA_Z2'])
df_belki['ILOSC1']=func_str_to_float(df_belki['ILOSC1'])
df_belki['ILOSC2']=func_str_to_float(df_belki['ILOSC2'])
df_belki['CENA_WAL']=func_str_to_float(df_belki['CENA_WAL'])
df_belki['CENA']=func_str_to_float(df_belki['CENA'])
df_belki=func_add_time(df_belki)
df_belki=func_correct_name(df_belki, 'NAZWA1')
#df=df.loc[df['ZNACZNIK']==19]

In [35]:
_=func_filtr_feler2(df, 'NAZWA1')

10202

10202it [05:32, 30.64it/s]
100%|█████████████████████████████████████████████████████████████████████████| 10202/10202 [00:00<00:00, 52128.23it/s]


,ZNACZNIK,NZNACZ,Z_DNIA,NAZWA,RODZAJ,NAZWA_IMP,TRZY,ILOSC,CENA_WAL
0,1,BELKI,2020-01-02,12668/MAR/B,290/V009 NET,GAP 12668 V005 MARKIZET 290CM,NET,6.8,5.30
1,8,OBRUSY,2020-01-02,CHO/GLEN/GRAN,35X140 831,35X140CM TWO LAYER TABLE RUNNER NAVY,831,1.0,3.05
2,15,LAMPY,2020-01-02,C/MANSI/LAMP/01,9X9X34 CZA+SRE 654,TABLE LAMP,654,1.0,4.15
3,5,CERAMIKA,2020-01-02,C/MAYRA1/MIS/1,27X27X10 GRA+ZŁO 792,9.8''ROUND BOWL DARK BLUE W/GOLD DECOR,792,1.0,7.88
4,10,ŚWIĘTA,2020-01-02,SW/OZD/VIONA/1-1,9X74CM B WSTĄŻK T12,NET ORNAMENT,T12,2.0,0.79
...,...,...,...,...,...,...,...,...,...
10197,3,ZAS.GOTOWE,2021-05-13,ZAS/MAGNUS/MIX,140X250 PRZE,MICROFIBER PRINTED QUILT HOT PRESS,514,1.0,2.40
10198,3,ZAS.GOTOWE,2021-05-13,ZAS/VELVET/BRĄZ,140X250 PRZEL,FRANCE VELVETN CURT.140X250 8RINGS BROWN,350,1.0,5.98
10199,19,ART.DEKORACYJNE,2021-05-13,ZESTAWUPOMIN.,T/19 XHF,DLA CIEBIE,XHF,1.0,0.00
10200,19,ART.DEKORACYJNE,2021-05-13,F/ZESTAWUPOMIN.,T/18,DLA CIEBIE,XHF,1.0,0.00


In [38]:

df_raport=func_raport(_, df_slownik, df_slownik_znacznik)

100%|█████████████████████████████████████████████████████████████████████████| 10202/10202 [00:00<00:00, 55920.91it/s]


In [59]:
_=func_filtr_feler2(df, 'NAZWA1')
df_raport=func_raport(_, df_slownik, df_slownik_znacznik)

13185

13185it [05:02, 43.63it/s]
100%|█████████████████████████████████████████████████████████████████████████| 13185/13185 [00:00<00:00, 23775.50it/s]


In [78]:
_belki=func_filtr_feler2(df_belki, 'NAZWA1')
df_raport_belki=func_raport(_belki, df_slownik, df_slownik_znacznik)

2190

2190it [02:07, 17.16it/s]
100%|███████████████████████████████████████████████████████████████████████████| 2190/2190 [00:00<00:00, 36095.42it/s]


In [90]:
df_belki[['ILOSC1','ILOSC2']].sum()

ILOSC1    192999.90
ILOSC2    193651.56
dtype: float64

In [89]:
_belki[['ILOSC1','ILOSC2']].sum()

ILOSC1    3782.08
ILOSC2    8028.83
dtype: float64

In [92]:
df_raport_belki.loc[df_raport_belki['NAZWA'].str.contains('193563/ZAS/J.S')].loc[df_raport_belki['Z_DNIA']==np.datetime64('2021-06-01')]

,ZNACZNIK,NZNACZ,Z_DNIA,NAZWA,RODZAJ,SYMBOL_IMP,NAZWA_IMP,TRZY,ILOSC,CENA_WAL
3,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,2.2,3.85
4,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,4.4,3.85
5,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,2.2,3.85
6,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,0.0,3.85


In [93]:
df_raport_belki.loc[df_raport_belki['ILOSC']>0]

,ZNACZNIK,NZNACZ,Z_DNIA,NAZWA,RODZAJ,SYMBOL_IMP,NAZWA_IMP,TRZY,ILOSC,CENA_WAL
0,1,BELKI,2021-06-01,F/193563/ZAS/CZA,280/HRT-38699,38700HRT-38699,POLYESTER FABRIC 280CM HRT-38699,HAR,1.80,3.75
1,1,BELKI,2021-06-01,F/193563/ZAS/CZA,280/HRT-38699,38700HRT-38699,POLYESTER FABRIC 280CM HRT-38699,HAR,1.80,3.75
2,1,BELKI,2021-06-01,193563/ZAS/J.RÓŻ,280/HRT-38707,38700HRT-38707,POLYESTER FABRIC 280CM HRT-38707,HAR,1.85,3.70
3,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,2.20,3.85
4,1,BELKI,2021-06-01,193563/ZAS/J.SRE,280/HRT-38717,38700HRT-38717,POLYESTER FABRIC 280CM HRT-38717,HAR,4.40,3.85
...,...,...,...,...,...,...,...,...,...,...
2183,1,BELKI,2022-05-27,7173/ŻORŻ/NATU,300/VBY004,7173 V110,VBY004 300CM WITH LB,EYL,2.15,3.20
2184,1,BELKI,2022-05-27,HAMILTON/DIM/111,320/BŁĘ+GRAN1 DIMOUT,HAMILTON 111,"320CM HAMILTON BLACKOUT 111, 247 GSM, DOUBLE SIDE",ADE,1.60,8.60
2185,1,BELKI,2022-05-27,VARNA/ZAS/POP+CZ,140 12Z LONETA,VARNA COL12ROJO,VARNA COL12 140CM ROJO LONETA,LLA,10.50,2.00
2187,1,BELKI,2022-05-30,1403/ZAS/LATTE,300/35-BEJ,ADA 35-BEJ,ADA FONLUK 195GSM COL 35-BEJ 300CM,KAR,3.00,2.40


In [95]:
df_raport.loc[df_raport['ILOSC']>0].append(df_raport_belki.loc[df_raport_belki['ILOSC']>0], ignore_index=True).to_excel('raport_felery/df_raport_felery_21_06_22_06.xlsx')

In [98]:
df_slownik.loc[df_slownik['NAZWA'].str.contains('VELVET/280/J.K')]#.loc[df_slownik['RODZAJ']=='140X180 EUR']

,KOD_TEMA,NAZWA,RODZAJ,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,SYMBOL_IMP
110844,224071,VELVET/280/J.K,280/ZF-00,1,783,FRENCH VELVET 300GSM 280CM COL.(ZF-00)ZF-68,"3,7",FRENCH ZF-68
117815,308282,F/VELVET/280/J.K,280/ZF-00,1,783,FRENCH VELVET 300 GSM 280CM COL.ZF-00,"3,56",FRENCH VELV.280
148201,335827,F/VELVET/280/J.K,280/ZF-00,1,783,FRENCH VELVET 300 GSM 280CM COL.ZF-00,"3,56",FRENCH VELV.280
191090,409891,F/VELVET/280/J.K,280/ZF-00,1,783,FRENCH VELVET 300GSM 280CM COL.(ZF-00)ZF-68,"3,7",FRENCH ZF-68


In [201]:
#_.merge(df_test[['Z_DNIA','name_without_f','RODZAJ','ILOSC','NAZWA', 'ZNACZNIK']], how='outer', left_on=['Z_DNIA','NAZWA1','RODZAJ1', 'ZNACZNIK'], right_on=['Z_DNIA','name_without_f','RODZAJ', 'ZNACZNIK'])

In [16]:
df_slownik=pd.read_csv('../słowniki/slownik.csv',
                       encoding='Cp1250',
                      sep="|")
df_slownik=df_slownik.loc[df_slownik['IS_DELETED']=='N'][['KOD_TEMA', 'NAZWA','RODZAJ','ZNACZNIK', 'TRZY', 'NAZWA_IMP', 'CENA_WAL', 'SYMBOL_IMP']]
df_slownik_znacznik=pd.read_csv('../słowniki/znacznik.csv',
                       encoding='Cp1250',
                      sep="|")
df_slownik_znacznik=df_slownik_znacznik.loc[df_slownik_znacznik['IS_DELETED']=='N'][['NAZWA','ZNACZNIK']]
df_slownik_znacznik=df_slownik_znacznik.rename(columns={'NAZWA':'NZNACZ'})

In [14]:
for i in df_slownik.columns:
    if 'ZNACZ' in i:
        print(i)

ZNACZNIK


In [17]:
df_slownik_znaczniknacznik

,RECNO,IS_DELETED,ZNACZNIK,NAZWA,OSOBA_ODP,KOD_KUPCA,TRANSPORT,MIN_UPUST,MAX_UPUST,KOLEJNY,DNI_PLATN,STATUS_I,KOD_DOST,PRODUCENT,KODPROD,MARZA_ZAL,MARZA_ZALH,BLOK_RABG,MARKA_WL,AKTYWNOSC,IE$0,TYP_PROD,TYP_PRODD,IKONA_ID,PODZIAL,IL_MIN_PH,IL_MIN_INT
0,72,N,53,WKŁ+PODUSZ.,0,0,0,,,0,0,,,,,0,0,,,,WKŁ+PODUSZ.,E,ŁÓŻKO,,D,0,0
1,69,N,90,PROPOZYCJE,0,0,0,,,0,0,,,,,0,0,,,,PROPOZYCJE,E,INNE,,D,0,0
2,29,N,1,BELKI,178,0,0,,,0,0,,,,,0,0,,1,,BELKI,A,OKNO,BELK,A,0,0
3,30,N,2,RĘCZNIKI,0,0,0,,,0,0,,,,,0,0,,5,,RĘCZNIKI,A,RĘCZNIKI,RE,B,0,0
4,31,N,3,ZAS.GOTOWE,614,0,0,,,0,0,,,,,0,0,,6,,ZAS.GOTOWE,A,OKNO,ZAS,A,0,0
5,32,N,4,NARZUTY,339,0,0,,,0,0,,,,,0,0,,9,,NARZUTY,A,ŁÓŻKO,NARZ,B,0,0
6,33,N,5,CERAMIKA,0,0,0,,,0,0,,,,,0,0,,3,,CERAMIKA,A,CERAMIKA+KWIATY,CER,B,0,0
7,34,N,6,POŚCIEL,0,0,0,,,0,0,,,,,0,0,,7,,POŚCIEL,A,ŁÓŻKO,POŚ,B,0,0
8,35,N,7,PODUSZKI,339,0,0,,,0,0,,,,,0,0,,0,,PODUSZKI,A,ŁÓŻKO,PODUSZ,B,0,0
9,36,N,8,OBRUSY,1,0,0,,,0,0,,,,,0,0,,2,,OBRUSY,A,STOŁ,OBRUS,B,0,0


In [179]:
_

,FAKTURA,Z_DNIA,NAZWA1,RODZAJ1,CENA_Z1,ILOSC1,NAZWA2,RODZAJ2,CENA_Z2,ILOSC2,KOD_TEMA,VAT_P,CENA,ZNACZNIK,TRZY,NAZWA_IMP,CENA_WAL,year,month,name_tmp,czy_z_f,name_len
245,SC5600021985,2020-01-08,F/K/GINA/01,43X31X18 B K5,23.42,0.0,F/K/GINA/01,43X31X18 B K5,23.42,2.0,321422,23,0.0,19,K5,43X31X18 BASKET WHITE + BLUE LACE,5.040,2020,1,K/GINA/01,tak,11
2886,SC5600022713,2020-02-27,F/GNIOTEK,8CM 134,1.23,0.0,F/GNIOTEK,8CM 134,1.23,23.0,355911,23,0.0,19,134,FACE BALL,0.295,2020,2,GNIOTEK,tak,9
4224,SC5600023066,2020-03-11,F/K/TINA/01,49X34X28X13X38 K7,21.66,0.0,F/K/TINA/01,49X34X28X13X38 K7,21.66,1.0,367072,23,0.0,19,K7,WOOD & WILLOW,4.080,2020,3,K/TINA/01,tak,11
5456,SC5600023353,2020-04-14,F/K/RENE/03,34X22X9 K7,10.40,0.0,F/K/RENE/03,34X22X9 K7,10.40,1.0,325647,23,0.0,19,K7,WILLOW,1.960,2020,4,K/RENE/03,tak,11
12434,SC5600025655,2020-10-08,F/K/GLORIA/02,37X25X31 B+NIEB,18.54,0.0,F/K/GLORIA/02,37X25X31 B+NIEB,18.54,1.0,321704,23,0.0,19,K5,37X25X31 BASKET WHITE+BLUE,3.990,2020,10,K/GLORIA/02,tak,13
12435,SC5600025655,2020-10-08,F/K/KOR/INA/2/B,18X16X7 SERCE,5.96,0.0,F/K/KOR/INA/2/B,18X16X7 SERCE,5.96,1.0,227015,23,0.0,19,388,18X16X7H HEART,1.360,2020,10,K/KOR/INA/2/B,tak,15
12436,SC5600025655,2020-10-08,F/K/KOR/INA/3/B,14X12X7 SERCE,5.09,0.0,F/K/KOR/INA/3/B,14X12X7 SERCE,5.09,1.0,227016,23,0.0,19,388,13.5X12X7H HEART,1.160,2020,10,K/KOR/INA/3/B,tak,15
12437,SC5600025655,2020-10-08,F/K/KOR/IZA/1/B,23X23X9 B,9.00,0.0,F/K/KOR/IZA/1/B,23X23X9 B,9.00,1.0,379202,23,0.0,19,388,WHITE/1 23X23X9,1.760,2020,10,K/KOR/IZA/1/B,tak,15
12438,SC5600025655,2020-10-08,F/K/KOR/MOLY/1/B,21RDX8,6.65,0.0,F/K/KOR/MOLY/1/B,21RDX8,6.65,1.0,227017,23,0.0,19,388,WHITE 23.5X8H ROUND,1.300,2020,10,K/KOR/MOLY/1/B,tak,16
12439,SC5600025655,2020-10-08,F/K/KOR/MOLY/2/B,24RDX8,8.69,0.0,F/K/KOR/MOLY/2/B,24RDX8,8.69,1.0,336081,23,0.0,19,388,23.5X8H ROUND,1.700,2020,10,K/KOR/MOLY/2/B,tak,16
